<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Audio_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 52.4 MB/s 
     |████████████████████████████████| 7.6 MB 43.7 MB/s 


In [2]:
! pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6q9lkgbg
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-6q9lkgbg
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175315 sha256=69df2ef50b92a34d57ed70f792abbeaf5b66aed7444df49a75f5b9fd60d2b55a
  Stored in directory: /tmp/pip-ephem-wheel-cache-egsfjq42/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper


In [3]:
import transformers
transformers.__version__

'4.25.1'

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Download youtube video

In [5]:
! pip install youtube-dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.9 MB/s 


In [6]:
from __future__ import unicode_literals
import youtube_dl


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    'outtmpl':"." + '/video.%(ext)s',
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=srSNZlbhCwk&t=3s'])
    
absolute_path = "video.wav" #file name of your downloaded audio

[youtube] srSNZlbhCwk: Downloading webpage
[download] Destination: ./video.webm
[download] 100% of 763.32KiB in 00:11
[ffmpeg] Destination: ./video.wav
Deleting original file ./video.webm (pass -k to keep)


## Transcribe text from Audio

In [7]:
import whisper
model = whisper.load_model("large")
options = dict(language='en', beam_size=5, best_of=5)
#transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)
translation = model.transcribe("/content/video.wav", **translate_options)

100%|██████████████████████████████████████| 2.87G/2.87G [00:22<00:00, 136MiB/s]
/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [3]:
translation

{'text': " This happened when I was 12 years old, I went to my grandmother's house with my mom My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there It was night and everyone went to attend the wedding in a nearby village except me Because I wanted to play games on the computer, then my uncle called me that his car has been punctured There is a tire in the house, we are close, come with the tire When I went to him, I saw that a strange thing was sitting, I asked him who are you Then he showed his face, which was very scary, then I noticed that he was just up to the stomach I ran immediately from there, but I fainted while running, when my eyes opened, I was at my grandmother's house I still don't know what it was",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 5.4,
   'text': " This happened when I was 12 years old, I went to my grandmother's house with my mom",
   'tokens': [639,
    2011,
    562,
    286

In [8]:
translation['text']

" This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was."

In [11]:
import joblib
joblib.dump(model,'transcribe.pkl')

['transcribe.pkl']

## Save the Automatic Speech Recognition Pipeline

In [ ]:
from transformers import pipeline
pipe = pipeline(task="automatic-speech-recognition", model="openai/whisper-large",max_length=5000)
pipe("/content/video.wav")

## Zeroshot classification using transformers

https://www.section.io/engineering-education/how-to-implement-zero-shot-classification-using-python/#:~:text=Zero-shot%20classification%20is%20a%20technique%20that%20allows%20us,topic%2C%20emotion%2C%20or%20event%20described%20by%20the%20label.

In [24]:
from transformers import pipeline
classifier_pipeline = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")

## Save the zeroshot pipeline

In [25]:
classifier_pipeline.save_pretrained("zeroshot")

In [27]:
text = """ This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was."""

In [29]:
new_pipeline = pipeline ("zero-shot-classification", model = "/content/zeroshot")
label_candidate = ['horror', 'comedy', 'sentimental','romantic','seroius']
new_pipeline(text, label_candidate)

{'sequence': " This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was.",
 'labels': ['horror', 'seroius', 'sentimental', 'comedy', 'romantic'],
 'scores': [0.6341521739959717,
  0.18177475035190582,
  0.12192074954509735,
  0.037401482462882996,
  0.02475087158381939]}

In [10]:
label_candidate = ['Positive', 'Negative', 'Neutral']
classifier_pipeline (translation['text'], label_candidate)

{'sequence': " This is when I was 12 years old, I once went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I started playing games as soon as I went there. It was night and everyone went to attend the wedding in a nearby village, except me. Because I had to play games on the computer, then my uncle called me that his car has a puncture. There is a tire in the house, we are nearby, come with the tire. When I reached him, I saw that a strange thing was sitting, I asked him who you are. So he showed his face, which was very scary, then I noticed that it was just up to the stomach. I ran away from there, but while running I fainted. When I woke up, I was at my grandmother's house, I still don't know what it was.",
 'labels': ['Negative', 'Positive', 'Neutral'],
 'scores': [0.520041286945343, 0.24548503756523132, 0.23447363078594208]}

## Huggingface whisper model

In [8]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration,WhisperFeatureExtractor

In [12]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
feature = WhisperFeatureExtractor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

In [3]:
newm = whisper.load_audio('/content/video.wav')

In [10]:
newm.shape

(731440,)

In [ ]:
translation = newm.transcribe("/content/Audio1.mp3", **translate_options)


In [18]:
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration,WhisperFeatureExtractor


processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

inputs = processor(np.array(newm), return_tensors="pt")
input_features = inputs.input_features

generated_ids = model.generate(inputs=input_features)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
transcription

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


' I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a young man, and I am a y

In [19]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Librosa can handle videos spanning upto 30 seconds

In [21]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

speech, _ = librosa.load("/content/video.wav")

processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "en", task = "transcribe")
input_features = processor(speech, return_tensors="pt").input_features 
predicted_ids = model.generate(input_features,max_length=1000)
transcription = processor.batch_decode(predicted_ids)

print(processor.batch_decode(predicted_ids, skip_special_tokens = True))

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[" This is when I was 12 years old, I went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I went there and started playing games. It was night and everyone went to attend the wedding in a nearby village except me, because I wanted to play games on the computer. Then my uncle called me and said that his car's"]


In [22]:
predicted_ids = model.generate(input_features,max_new_tokens=3000)
transcription = processor.batch_decode(predicted_ids)

print(processor.batch_decode(predicted_ids, skip_special_tokens = True))

[" This is when I was 12 years old, I went to my grandmother's house with my mom. My uncle had a computer at that time, which had a lot of games, so I went there and started playing games. It was night and everyone went to attend the wedding in a nearby village except me, because I wanted to play games on the computer. Then my uncle called me and said that his car's"]
